In [3]:
from transpile_benchy.metrics.gate_counts import DepthMetric, TotalMetric, TotalSwaps
from qiskit.circuit.library import iSwapGate
from qiskit.transpiler import CouplingMap
from mirror_gates.pass_managers import Mirage, QiskitLevel3
from mirror_gates.utilities import SubsMetric
from mirror_gates.logging import transpile_benchy_logger

In [4]:
from itertools import product

module_size = 4
children = 4
total_levels = 3
qubit_counter = 0
coupling_map = []  # TODO


def get_module(qubit_counter, starting_qubit=None):
    temp_qubits = []
    if starting_qubit is not None:
        temp_qubits.append(starting_qubit)
    for i in range(module_size - len(temp_qubits)):
        temp_qubits.append(qubit_counter)
        qubit_counter += 1
    temp_edges = [el for el in product(temp_qubits, repeat=2) if el[0] != el[1]]
    # remove before returning so only get the qubits specific to the new module
    if starting_qubit is not None:
        temp_qubits.remove(starting_qubit)
    return qubit_counter, temp_qubits, temp_edges


qubit_counter, root_q, root_e = get_module(qubit_counter)
coupling_map.extend(root_e)


def foo_recursive(qubit_counter, root_q, level):
    # convention, root only counts as level=1
    if level == (total_levels - 1):
        return qubit_counter
    offset = 1 if level != 0 else 0
    # this offset says that if we aren't the first level then we need less one child bc it points to the parent
    for node_i in range(children - offset):
        node_q = root_q[node_i]  # +offset]
        # extend a module from this node
        qubit_counter, child_q, child_e = get_module(
            qubit_counter, starting_qubit=node_q
        )
        coupling_map.extend(child_e)

        # XXX
        qubit_counter = foo_recursive(qubit_counter, child_q, level + 1)
    return qubit_counter


##
qubit_counter = foo_recursive(qubit_counter, root_q, level=0)
qubits = list(range(qubit_counter))

# finally,
# delete repeats
coupling_map = list(set(coupling_map))
# add bidirectional edges
coupling_map += [[q2, q1] for q1, q2 in coupling_map]
# check no edges to self
coupling_map = [[q1, q2] for q1, q2 in coupling_map if q1 != q2]

tree = CouplingMap(coupling_map)

In [5]:
# N = 4
# coupling_map = CouplingMap.from_line(N)
# coupling_map = CouplingMap.from_heavy_hex(5)
square = CouplingMap.from_grid(6, 6)

In [6]:
from transpile_benchy.library import CircuitLibrary

library = CircuitLibrary.from_txt("../../circuits/medium_circuits.txt")
# library = CircuitLibrary.from_txt("../../circuits/debug.txt")

In [7]:
# XXX set consolidate to False
# this is allowed only because my pass manager will preserve consolidation
# see post_stage, I call fastconsolidate manually

# NOTE: use TotalSwaps to verify baselines have > 0 swaps
# otherwise, there is no room for improvement.
# we can include these if we want to show our methods will still work
# but somewhat trivial since we just append VF2Layout
metrics = [
    DepthMetric(consolidate=False),
    TotalMetric(consolidate=False),
    TotalSwaps(consolidate=False),
    SubsMetric(),
]

transpilers = [
    QiskitLevel3(square, name="Qiskit-square"),
    Mirage(square, name="Mirage-square"),
    QiskitLevel3(tree, name="Qiskit-tree"),
    Mirage(tree, name="Mirage-tree"),
]

In [8]:
from transpile_benchy.benchmark import Benchmark

benchmark = Benchmark(
    transpilers=transpilers,
    circuit_library=library,
    metrics=metrics,
    logger=transpile_benchy_logger,
    num_runs=3,
)

benchmark.run()
# print(benchmark)

INFO:transpile_benchy:Running benchmarks for circuits...


Percent volume of A_C2	 | Cost	 | Sequence name
  0.00% =    0/1    	 | 0.00	 | 
  0.00% =    0/1    	 | 0.50	 | xx_plus_yy(-1.57080)
 50.00% =    1/2    	 | 1.00	 | xx_plus_yy(-1.57080).xx_plus_yy(-1.57080)
100.00% =    1/1    	 | 1.50	 | xx_plus_yy(-1.57080).xx_plus_yy(-1.57080).xx_plus_yy(-1.57080)
Percent volume of A_C2	 | Cost	 | Sequence name
  0.00% =    0/1    	 | 0.00	 | 
  0.00% =    0/1    	 | 1.00	 | cx
  0.00% =    0/1    	 | 2.00	 | cx.cx
100.00% =    1/1    	 | 3.00	 | cx.cx.cx


Circuits from library:   0%|          | 0/15 [00:00<?, ?it/s]INFO:transpile_benchy:Running benchmark for circuit qec9xz_n17


Loading qec9xz_n17 from QASMBench


Circuits from library:   7%|▋         | 1/15 [01:28<20:34, 88.18s/it]INFO:transpile_benchy:Running benchmark for circuit seca_n11


...

Loading multiplier_n15 from QASMBench


Circuits from library:  47%|████▋     | 7/15 [17:25<24:08, 181.05s/it]INFO:transpile_benchy:Running benchmark for circuit qft_n18


Loading qft_n18 from QASMBench


Circuits from library:  53%|█████▎    | 8/15 [22:04<24:46, 212.32s/it]INFO:transpile_benchy:Running benchmark for circuit sat_n11


Loading sat_n11 from QASMBench


In [ ]:
# print(benchmark)

In [ ]:
benchmark.summary_statistics(transpilers[1], transpilers[3])

In [ ]:
from transpile_benchy.render import plot_benchmark

plot_benchmark(
    benchmark, save=1, legend_show=1, filename="grid", color_override=[0, 3, 6, 7]
)